# Amazon Web Scrapping with Selenium and Beautiful Soup

In [1]:
#Importing libraries
import csv
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

#Get url based on search term and Navigating to next page
def get_url(search_term):
    template='https://www.amazon.com/s?k={}&crid=25VEVPPHRCLCX'
    search_term=search_term.replace(' ','+')
    url=template.format(search_term)
    url+='&page={}'
    return url

#Extracting each search item as a dataframe within a method
def extract_record(item):
    atag=item.h2.a
    description=atag.text.strip()
    url='https://www.amazon.com'+atag.get('href')
    try:
        price_parent=item.find('span','a-price')
        price=price_parent.find('span','a-offscreen').text
    except AttributeError:
            return
    try:
        rating=item.i.text
        review_count=item.find('span',{'class':'a-size-base'}).text
    except AttributeError:
        rating=''
        review_count=0
    df=pd.DataFrame(columns=['Description','price','rating','review_count','url'])
    df=df.append({'Description': description,  'price': price,'rating':rating,'review_count':review_count,'url':url}, ignore_index=True)
    return df

#Main function initiate webdriver and invoke methods
def main(search_term):
    """Run main program"""
    #Start webdriver
    driver=webdriver.Chrome('C://Users//Hp//Downloads//chromedriver_win32 (1)//chromedriver.exe')
    url=get_url(search_term)
    df=pd.DataFrame(columns=['Description','price','rating','review_count','url'])
    for page in range(1,21):
        driver.get(url.format(page))
        soup=BeautifulSoup(driver.page_source,'html.parser')
        results=soup.find_all('div',{'data-component-type':'s-search-result'})
        for item in results:
            df=df.append(extract_record(item))
    driver.close()
    
    #Saving results to csv file
    df.to_csv("results.csv",encoding='utf-8',header=True,index=False)
       

In [2]:
main('mobile phones')